## Presidents and Income
### An intro to python
In this notebook we'll use historical data to look at the relationships between presidential parties and income growth.

The first step is to import the libraries we need for the analysis.

In [1]:
import pandas as pd #data science library for dataframes and data manipulation
import os #operating system library to set the working directory

In [2]:
# setting the working directory locally. This is a step you need to change
os.chdir("/Users/bi6fgjk/Documents/pers/py_study") 

Pandas has a read_csv function that can read files directly from Github. This is convenient since it saves people from needing to download the files in order to follow along. 

I made a csv of table F-1: https://www.census.gov/data/tables/time-series/demo/income-poverty/historical-income-families.html and saved it to Github to make it available for download. 

All values are in 2019 dollars as provided by the Census.

Footnotes to the data are available here: https://www.census.gov/topics/income-poverty/income/guidance/cps-historic-footnotes.html

In [3]:
hist_df = pd.read_csv("https://raw.githubusercontent.com/natekratzer/hist_income_by_quintile/main/hist_income47.csv")
hist_df.tail(8) #checking the last 8 rows

,Year,20,40,60,80,95
67,1954,"16,759","29,479","39,907","54,676","86,645"
68,1953,"17,976","30,507","40,936","55,130","85,738"
69,1952 (7),"17,370","28,098","38,014","51,416","79,996"
70,1951,"16,826","27,432","36,289","49,548","77,652"
71,1950,"15,441","26,550","35,335","49,112","80,088"
72,1949 (6),"14,523","24,811","33,471","47,298","75,921"
73,1948,"15,385","25,295","33,773","47,114","76,722"
74,1947 (5),"15,903","25,662","34,799","49,377","81,043"


In [4]:
# suppose I want to inspect the data more
# an easy way is to write a quick .csv file and then open it still in Jupyter Lab
hist_df.to_csv("explore_my_data.csv")

The year column has footnotes in it that are marked off with parentheses.
So I googled 'Regular Expression to remove everything in parentheses'. 

In [5]:
hist_df['Year'] = hist_df['Year'].str.replace(r"\(.*\)", "")
hist_df.head()

,Year,20,40,60,80,95
0,2019,"40,000","69,000","105,038","164,930","304,153"
1,2018,"36,514","64,165","98,085","152,837","284,299"
2,2017,"35,252","62,960","97,523","153,979","284,244"
3,2017,"34,992","62,610","96,324","151,622","272,736"
4,2016,"34,518","61,732","95,637","148,691","267,602"


In [6]:
def comma_replace(x):
    x = x.str.replace(',', '')
    return x

In [7]:
hist_df = hist_df.apply(comma_replace)
hist_df.head()

,Year,20,40,60,80,95
0,2019,40000,69000,105038,164930,304153
1,2018,36514,64165,98085,152837,284299
2,2017,35252,62960,97523,153979,284244
3,2017,34992,62610,96324,151622,272736
4,2016,34518,61732,95637,148691,267602


Dictionaries are a core Python type, and they are used here to map each column name onto the type of data in that column. 

In [8]:
type_dict = {
    'Year' : 'datetime64[ns]',
    '20'   : 'int32',
    '40'   : 'int32',
    '60'   : 'int32',
    '80'   : 'int32',
    '95'   : 'int32'
}

In [9]:
type_dict

{'Year': 'datetime64[ns]',
 '20': 'int32',
 '40': 'int32',
 '60': 'int32',
 '80': 'int32',
 '95': 'int32'}

In [10]:
hist_df = hist_df.astype(type_dict)
hist_df['add 20 and 40'] = hist_df['20'] + hist_df['40'] #to show it's numeric
hist_df.head()

,Year,20,40,60,80,95,add 20 and 40
0,2019-01-01,40000,69000,105038,164930,304153,109000
1,2018-01-01,36514,64165,98085,152837,284299,100679
2,2017-01-01,35252,62960,97523,153979,284244,98212
3,2017-01-01,34992,62610,96324,151622,272736,97602
4,2016-01-01,34518,61732,95637,148691,267602,96250


In [11]:
hist_df = hist_df.drop(columns = ['add 20 and 40'])
hist_df.head()

,Year,20,40,60,80,95
0,2019-01-01,40000,69000,105038,164930,304153
1,2018-01-01,36514,64165,98085,152837,284299
2,2017-01-01,35252,62960,97523,153979,284244
3,2017-01-01,34992,62610,96324,151622,272736
4,2016-01-01,34518,61732,95637,148691,267602


In [12]:
hist_df = hist_df.groupby(['Year']).mean() #have to call mean before taking percent change, not sure why?
hist_df = hist_df.pct_change()
hist_df.tail()

,20,40,60,80,95
Year,,,,,
2015-01-01,0.039833,0.049031,0.050342,0.033252,0.038025
2016-01-01,0.055306,0.033051,0.026831,0.031946,0.036779
2017-01-01,0.017498,0.017058,0.013452,0.027638,0.040687
2018-01-01,0.039633,0.021980,0.011984,0.000239,0.020859
2019-01-01,0.095470,0.075353,0.070887,0.079124,0.069835


In [13]:
hist_df.head()

,20,40,60,80,95
Year,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN
1948-01-01,-0.032572,-0.014301,-0.029484,-0.045831,-0.053317
1949-01-01,-0.056029,-0.019134,-0.008942,0.003905,-0.010440
1950-01-01,0.063210,0.070090,0.055690,0.038353,0.054886
1951-01-01,0.089696,0.033220,0.026999,0.008878,-0.030417


In [14]:
hist_df = hist_df.dropna()

def mult100(x):
    x = x * 100
    return x

# hist_df = hist_df.apply(lambda x: x * 100)
hist_df = hist_df.apply(mult100)
hist_df.tail()

,20,40,60,80,95
Year,,,,,
2015-01-01,3.983342,4.903097,5.034170,3.325182,3.802473
2016-01-01,5.530588,3.305052,2.683115,3.194576,3.677904
2017-01-01,1.749812,1.705760,1.345191,2.763785,4.068729
2018-01-01,3.963328,2.197977,1.198368,0.023887,2.085892
2019-01-01,9.547023,7.535261,7.088750,7.912351,6.983493
